In [1]:
%load_ext autoreload
%autoreload 2
import sys
import pandas as pd
import os
import matplotlib.pyplot as plt
import cortex
import seaborn as sns
from os.path import join
from collections import defaultdict
import numpy as np
from sklearn.preprocessing import StandardScaler
import joblib
import dvu
import sys
sys.path.append('../notebooks')
from tqdm import tqdm
from sasc.config import FMRI_DIR, STORIES_DIR, RESULTS_DIR
from neuro.config import repo_dir, PROCESSED_DIR
from neuro.features.qa_questions import get_questions, get_merged_questions_v3_boostexamples
import sasc.viz

### S02 comparisons

In [2]:
gemv_flatmaps_pilot = joblib.load(join(
    RESULTS_DIR, "processed", "flatmaps", 'resps_avg_dict_pilot.pkl'))
gemv_flatmaps_pilot5 = joblib.load(join(
    RESULTS_DIR, "processed", "flatmaps", 'resps_avg_dict_pilot5.pkl'))
gemv_flatmaps_dict = gemv_flatmaps_pilot | gemv_flatmaps_pilot5

In [ ]:
gemv_flatmaps_dict.keys()

normalize_flatmaps = True
if normalize_flatmaps:
    for k, v in gemv_flatmaps_dict.items():
        flatmap_unnormalized = gemv_flatmaps_dict[k]
        gemv_flatmaps_dict[k] = (
            flatmap_unnormalized - flatmap_unnormalized.mean()) / flatmap_unnormalized.std()

In [7]:
# subject = 'S02'
subject = 'S02'
rois = joblib.load(join(FMRI_DIR, 'brain_tune/voxel_neighbors_and_pcs/',
                   f'communication_rois_v2_UT{subject}.jbl'))
rois_dict_raw = {i: rois[i] for i in range(len(rois))}

# custom merge contralateral regions
if subject == 'S02':
    raw_idxs = [
        [0, 7],
        [3, 4],
        [1, 5],
        [2, 6],
    ]
elif subject == 'S03':
    raw_idxs = [
        [0, 7],
        [3, 4],
        [2, 5],
        [1, 6],
    ]
rois_dict = {
    i: np.vstack([rois_dict_raw[j] for j in idxs]).sum(axis=0)
    for i, idxs in enumerate(raw_idxs)
}

In [8]:
gemv_flatmaps_dict.keys()
avg_defaultdict = defaultdict(list)
for roi_idx, roi in rois_dict.items():
    for explanation in gemv_flatmaps_dict.keys():
        avg_defaultdict[roi_idx].append(
            np.mean(gemv_flatmaps_dict[explanation][roi > 0]))
        # corrs_defaultdict[roi_idx].append(
        # np.corrcoef(roi, gemv_flatmaps_dict[explanation])[0, 1])

In [9]:
df = pd.DataFrame(avg_defaultdict, index=gemv_flatmaps_dict.keys())
df.columns = [f'ROI_{i}' for i in rois_dict.keys()]
df
# add mean col
df['AVG'] = df.mean(axis=1)
df = df.round(3).sort_values('AVG', ascending=False)
# display pandas formatting with redblue colormap centered at 0 rounded to 3 decimal places
vabs = max(abs(df.values.min()), abs(df.values.max()))
df.style.background_gradient(cmap='coolwarm', axis=None, vmax=vabs, vmin=-vabs).format("{:.3f}").set_caption(
    'Average GEM-V driving response averaged over ROI'
)

,,ROI_0,ROI_1,ROI_2,ROI_3,AVG
dialogue,qa,0.511,1.121,1.144,1.085,0.965
communication,299,0.891,0.828,1.114,0.934,0.941
rejection,342,0.736,0.744,1.170,0.766,0.854
laughter,426,1.035,1.001,0.689,0.567,0.823
negations,qa,0.099,0.431,0.828,1.025,0.596
personal or interactions interactions,qa,0.785,0.619,0.232,0.352,0.497
cultural references,qa,-0.155,0.446,0.854,0.841,0.497
personal reflections or thoughts,qa,1.479,0.612,0.016,-0.457,0.413
negativity,248,0.209,0.396,0.366,0.651,0.405
time,212,-0.168,0.213,0.411,1.153,0.402


In [47]:
sasc.viz._save_flatmap(
    gemv_flatmaps_dict[('relationships between people', 'qa')], subject, fname_save=f'gemv_flatmaps/relationships_{subject}.png')

/home/chansingh/automated-explanations/sasc/viz.py:39: DeprecationWarning: invalid escape sequence '\s'
  def imshow_diverging(mat, clab="Mean response ($\sigma$)", clab_size='medium', vabs_multiplier=1):
/home/chansingh/automated-explanations/sasc/viz.py:312: DeprecationWarning: invalid escape sequence '\s'
  f'"{expl_voxel}"\nvoxel\nresponse ($\sigma_f$)', fontsize='x-small'
/home/chansingh/automated-explanations/sasc/viz.py:357: DeprecationWarning: invalid escape sequence '\s'
  plt.ylabel('Mean voxel response ($\sigma_f$)')
/home/chansingh/automated-explanations/sasc/viz.py:452: DeprecationWarning: invalid escape sequence '\s'
  plt.ylabel('Mean voxel response ($\sigma_f$)')
/home/chansingh/automated-explanations/sasc/viz.py:513: DeprecationWarning: invalid escape sequence '\s'
  plt.ylabel('Mean voxel response ($\sigma$)')
/home/chansingh/automated-explanations/sasc/viz.py:548: DeprecationWarning: invalid escape sequence '\s'
  plt.ylabel('Mean voxel response ($\sigma_f$)')
/home/